This notebook created some data for the DMS model via the REST API.

# Contents
- [Schema and data](#Schema-and-data)
- [Inserting and retrieving](#Inserting-and-retrieving)
- [Wildcards](#Wildcards)
- [Filtering](#Filtering)
- [Visualization](#Visualization)


In [28]:
import requests
from rich.pretty import pprint
from rich import print_json
import json

# location of the DB (B=base URL for MongoDB, DB=database name)
B='http://localhost:8080'
DB='havlasek0'

# small wrapper for the REST API
def get(p,no_show=False,**kw):
    r=requests.get(f'{B}/{DB}/{p}',params=kw)
    if not r.ok: raise RuntimeError(r.text)
    data=json.loads(r.text)
    if not no_show: pprint(data)
    return data


# HELP: how to do a POST with both data (json) and query parameters?
def post(p,**kw):
    # pprint(kw)
    r=requests.post(f'{B}/{DB}/{p}',json=kw)
    if not r.ok:
        raise RuntimeError(r.text)
    return json.loads(r.text)
def patch(p,**kw):
    r=requests.patch(f'{B}/{DB}/{p}',json=kw)
    if not r.ok: raise RuntimeError(r.text)
    return json.loads(r.text)



# Set database schema

In [55]:
schema={
	"Material_moisture_sorption_isotherm": {
		"uh": {},
		"vg_b": {},
		"vg_m": {},
		"rhodry": {"unit": "kg/m3"}
	},
	"Material_moisture_permeability": {
		"mu": {},
		"Abs": {"unit": "kg/m2/day^(1/2)"},
		"capil_coef_power": {}
	},
	"Material_moisture": {
		"surface_factor": {"unit": "kg/m2/day"},
		"sorption_isotherm": {"link": "Material_moisture_sorption_isotherm"},
		"permeability": {"link": "Material_moisture_permeability"}
	},
	"Material_structural_basic_creep": {
		"realmatage": {"unit": "day"},
		"q1": {"unit": "MPa-1"},
		"q2": {"unit": "MPa-1"},
		"q3": {"unit": "MPa-1"},
		"q4": {"unit": "MPa-1"}
	},
	"Material_structural_drying_creep": {
		"p_tilde": {},
    	"k3": {},
		"khc": {},
		"alphaE": {},
		"alphaR": {},
		"alphaS": {}
	},
	"Material_structural_shrinkage": {
		"ksh": {},
		"timedependent_ksh": {"dtype": "?"}
	},
	"Material_structural_shrinkage_reduction": {
		"h_transition": {},
		"ksh_red": {}
	},
	"Material_structural_tensile_fracture": {
		"ft": {"unit": "MPa"},
		"Gf": {"unit": "MN/m"}
	},
	"Material_structural": {
		"basic_creep": {"link": "Material_structural_basic_creep"},
		"drying_creep": {"link": "Material_structural_drying_creep"},
		"shrinkage": {"link": "Material_structural_shrinkage"},
		"shrinkage_reduction": {"link": "Material_structural_shrinkage_reduction"},
		"tensile_fracture": {"link": "Material_structural_tensile_fracture"}
	},
	"Material": {
		"rho": {"unit": "MN/m3"},
		"moisture": {"link": "Material_moisture"},
		"structural": {"link": "Material_structural"}
	},

	"BeamLoadState_structural": {
		"F": { "unit": "MN" },
		"a": { "unit": "m" }
	},
	"BeamLoadState_moisture":{
		"drying_top": { "dtype": "?" },
		"drying_bottom": { "dtype": "?" },
		"initial_humidity": {}
	},
	"BeamLoadState": {
		"structural": {"link": "BeamLoadState_structural"},
		"moisture":{"link": "BeamLoadState_moisture"}
	},

	"BeamGeometry":{
		"length": { "unit": "m" },
		"height": { "unit": "m" },
		"breadth": { "unit": "m" }
	},

	"BeamModel_params": {
		"div": {"link": "BeamModel_params_div"},
		"height_elem_size": {"unit": "m"}
	},
	"BeamModel_params_div": {
        "length": {},
		"height": {}
	},
	"BeamModel": {
		"length": { "unit": "m" },
		"tm": {"link": "BeamModel_params"},
		"sm": {"link": "BeamModel_params"}
	},

	"BeamState": {
		"geometry": {"link": "BeamGeometry"},
		"params": {}
	},

	"Simulation":{
		"dt_init": { "unit": "day" },
		"dt_max": { "unit": "day" },
		"time_k": { "unit": "" },
		"time_end": { "unit": "day" }
	}
}
# don't know how to POST schema with force=True as query parameter...
# without force=True, this will work only once for a clean DB without schema
post('schema',**schema)

{
│   'Material_moisture_sorption_isotherm': {'uh': {}, 'vg_b': {}, 'vg_m': {}, 'rhodry': {'unit': 'kg/m3'}},
│   'Material_moisture_permeability': {'mu': {}, 'Abs': {'unit': 'kg/m2/day^(1/2)'}, 'capil_coef_power': {}},
│   'Material_moisture': {
│   │   'surface_factor': {'unit': 'kg/m2/day'},
│   │   'sorption_isotherm': {'link': 'Material_moisture_sorption_isotherm'},
│   │   'permeability': {'link': 'Material_moisture_permeability'}
│   },
│   'Material_structural_basic_creep': {
│   │   'realmatage': {'unit': 'day'},
│   │   'q1': {'unit': 'MPa-1'},
│   │   'q2': {'unit': 'MPa-1'},
│   │   'q3': {'unit': 'MPa-1'},
│   │   'q4': {'unit': 'MPa-1'}
│   },
│   'Material_structural_drying_creep': {
│   │   'p_tilde': {},
│   │   'k3': {},
│   │   'khc': {},
│   │   'alphaE': {},
│   │   'alphaR': {},
│   │   'alphaS': {}
│   },
│   'Material_structural_shrinkage': {'ksh': {}, 'timedependent_ksh': {'dtype': '?'}},
│   'Material_structural_shrinkage_reduction': {'h_transition': {}, 'ksh_red': {}},
│   'Material_structural_tensile_fracture': {'ft': {'unit': 'MPa'}, 'Gf': {'unit': 'MN/m'}},
│   'Material_structural': {
│   │   'basic_creep': {'link': 'Material_structural_basic_creep'},
│   │   'drying_creep': {'link': 'Material_structural_drying_creep'},
│   │   'shrinkage': {'link': 'Material_structural_shrinkage'},
│   │   'shrinkage_reduction': {'link': 'Material_structural_shrinkage_reduction'},
│   │   'tensile_fracture': {'link': 'Material_structural_tensile_fracture'}
│   },
│   'Material': {
│   │   'rho': {'unit': 'MN/m3'},
│   │   'moisture': {'link': 'Material_moisture'},
│   │   'structural': {'link': 'Material_structural'}
│   },
│   'BeamLoadState_structural': {'F': {'unit': 'MN'}, 'a': {'unit': 'm'}},
│   'BeamLoadState_moisture': {
│   │   'drying_top': {'dtype': '?'},
│   │   'drying_bottom': {'dtype': '?'},
│   │   'initial_humidity': {}
│   },
│   'BeamLoadState': {
│   │   'structural': {'link': 'BeamLoadState_structural'},
│   │   'moisture': {'link': 'BeamLoadState_moisture'}
│   },
│   'BeamGeometry': {'length': {'unit': 'm'}, 'height': {'unit': 'm'}, 'breadth': {'unit': 'm'}},
│   'BeamModel_params': {'div': {'link': 'BeamModel_params_div'}, 'height_elem_size': {'unit': 'm'}},
│   'BeamModel_params_div': {'length': {}, 'height': {}},
│   'BeamModel': {
│   │   'length': {'unit': 'm'},
│   │   'tm': {'link': 'BeamModel_params'},
│   │   'sm': {'link': 'BeamModel_params'}
│   },
│   'BeamState': {'geometry': {'link': 'BeamGeometry'}, 'params': {}},
│   'Simulation': {
│   │   'dt_init': {'unit': 'day'},
│   │   'dt_max': {'unit': 'day'},
│   │   'time_k': {'unit': ''},
│   │   'time_end': {'unit': 'day'}
│   }
}

RuntimeError: {"type":"ValueError","message":"Schema already defined (use force=True if you are sure).","url":"http://localhost:8080/havlasek0/schema","method":"POST","traceback":["Traceback (most recent call last):","  File '/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py', line 162, in __call__","    await self.app(scope, receive, _send)","  File '/usr/local/lib/python3.10/dist-packages/starlette/middleware/exceptions.py', line 75, in __call__","    raise exc","  File '/usr/local/lib/python3.10/dist-packages/starlette/middleware/exceptions.py', line 64, in __call__","    await self.app(scope, receive, sender)","  File '/usr/local/lib/python3.10/dist-packages/fastapi/middleware/asyncexitstack.py', line 21, in __call__","    raise e","  File '/usr/local/lib/python3.10/dist-packages/fastapi/middleware/asyncexitstack.py', line 18, in __call__","    await self.app(scope, receive, send)","  File '/usr/local/lib/python3.10/dist-packages/starlette/routing.py', line 680

In [ ]:
dta={ #BeamGeometry
    "length": {"value": 1.75, "unit": "m" },
    "height": {"value": 0.05, "unit": "m" },
    "breadth": {"value": 0.1, "unit": "m" }
}
ID=post('BeamGeometry',**dta)

{
│   'length': {'value': 1.75, 'unit': 'm'},
│   'height': {'value': 0.05, 'unit': 'm'},
│   'breadth': {'value': 0.1, 'unit': 'm'}
}